In [117]:
#Install the EPANET-Python toolkit (this is so we can process the hydraulic information)
#note that this takes a long time to run, if you want shorter trials (less accurate results) loosen the tolerance on the 
#while pointsInSimplex term

from epyt import epanet
import numpy as np
import pandas as pd

# Pull in relevant EPANET file
d = epanet('C:/Users/apgi227/OneDrive - University of Kentucky/Documents/MATLAB/DecemberEditsWhitesburg.inp');

# Let's Extend The Box-Complex to the tank scenario
# Random Positive Demand Factors
demandFactors = [[2, 0.5, 3], [4, 1, 2], [3, 1.5, 0.5], [5, 1.2, 1]]

# Because I am only testing one hour and epanet requires demand factors for all 24 hours
# I will create 23 dummy factors
dummyPattern = [1] * 23

# Now here are the three functions (tanks) that we need to optimize
RealTank1 = 1484.99
RealTank2 = 1411.48
RealTank3 = 1468.64

# Initialize the error matrix (four points in box complex and their respective error)
pointsInSimplex = [1, 1, 1, 1]

# This starts the box-complex, here I specify that all of the points need to have a very small error term before it is finished.
# Essentially, converge on the solution. However this may run for a long time and loosening the tolerance will allow for quicker runs
while pointsInSimplex[0] >= 0.0000001 or pointsInSimplex[1] >= 0.0000001 or pointsInSimplex[2] <= 0.0000001 or pointsInSimplex[3] >= 0.0000001:
    # Evaluation of error for each of the four points in the simplex
    for i in range(4):
        # Create the demand patterns to place in the simulation
        pattern1 = demandFactors[i][0]
        pattern2 = demandFactors[i][1]
        pattern3 = demandFactors[i][2]
        
        # The fourth demand factor is determined by the first three - this is the total demand function.
        # This is the mass balance in the system which may be found by taking the basedemand for each of the points that are
        # specified by a certain demand pattern - multiplying those base demands by that factor, and summing them altogether.
        # This gives total demand. To satisfy conservation of mass, the fourth demand factor is determined by the other three.
        # 3, 226, and 48 represent the summed base demands for their respective zones. 556 represents the total demand of that specific hour.
        pattern4 = (556 - (3 * pattern1) - (226 * pattern2) - (48 * pattern3)) / 18
        
        # Run an initial sim to initialize the box complex
        d.setPattern(1, [pattern1] + dummyPattern)
        d.setPattern(2, [pattern2] + dummyPattern)
        d.setPattern(3, [pattern3] + dummyPattern)
        d.setPattern(4, [pattern4] + dummyPattern)
        d.openHydraulicAnalysis()
        d.initializeHydraulicAnalysis()
        
        # Run and close analysis
        Series = d.getComputedHydraulicTimeSeries()
        d.closeHydraulicAnalysis()
        ModelTank1 = Series.Head[2, 302]
        ModelTank2 = Series.Head[2, 307]
        ModelTank3 = Series.Head[2, 304]
        Error = ((ModelTank1 - RealTank1) ** 2) + ((ModelTank2 - RealTank2) ** 2) + ((ModelTank3 - RealTank3) ** 2)
        pointsInSimplex[i] = Error
        
    if pointsInSimplex[0] <= 0.0000001 or pointsInSimplex[1] <= 0.0000001 or pointsInSimplex[2] <= 0.0000001 or pointsInSimplex[3] <= 0.0000001:
        break
    else:
        # Find the worst point and create the centroid of the remaining points
        maxVal, whereMax = max(pointsInSimplex), pointsInSimplex.index(max(pointsInSimplex))
        logical = [i for i in range(4) if pointsInSimplex[i] != maxVal]
        
        ph = demandFactors[whereMax]
        centroid = [(demandFactors[logical[0]][j] + demandFactors[logical[1]][j] + demandFactors[logical[2]][j]) / 3 for j in range(3)]
       
        # Expand the worst point over the centroid of the remaining points
        newPoint = [(2.5 * centroid[j]) - (1.5 * ph[j]) for j in range(3)]
        
        # Contract if the New Point is less than 0
        while newPoint[0] < 0 or newPoint[1] < 0 or newPoint[2] < 0:
            newPoint = [(0.5 * newPoint[j]) + (0.5 * centroid[j]) for j in range(3)]
        
        # Now evaluate the new point to see if it is better than the old point
        pattern1 = newPoint[0]
        pattern2 = newPoint[1]
        pattern3 = newPoint[2]
        pattern4 = (556 - (3 * pattern1) - (226 * pattern2) - (48 * pattern3)) / 18
        d.setPattern(1, [pattern1] + dummyPattern)
        d.setPattern(2, [pattern2] + dummyPattern)
        d.setPattern(3, [pattern3] + dummyPattern)
        d.setPattern(4, [pattern4] + dummyPattern)
        d.openHydraulicAnalysis()
        d.initializeHydraulicAnalysis()
        
        # Run and close analysis
        Series = d.getComputedHydraulicTimeSeries()
        d.closeHydraulicAnalysis()
        ModelTank1 = Series.Head[2, 302]
        ModelTank2 = Series.Head[2, 307]
        ModelTank3 = Series.Head[2, 304]
        Error = ((ModelTank1 - RealTank1) ** 2) + ((ModelTank2 - RealTank2) ** 2) + ((ModelTank3 - RealTank3) ** 2)
        
        # If the new error is less than the old store the value
        if Error < maxVal:
            demandFactors[whereMax] = [pattern1, pattern2, pattern3]
            pointsInSimplex[whereMax] = Error
            
        # If the error is worse than the new point, contract the worst point towards the centroid
        else:
            newPoint = [(0.5 * ph[j]) + (0.5 * centroid[j]) for j in range(3)]
            
            # Evaluate the new point
            pattern1 = newPoint[0]
            pattern2 = newPoint[1]
            pattern3 = newPoint[2]
            pattern4 = (556 - (3 * pattern1) - (226 * pattern2) - (48 * pattern3)) / 18
            d.setPattern(1, [pattern1] + dummyPattern)
            d.setPattern(2, [pattern2] + dummyPattern)
            d.setPattern(3, [pattern3] + dummyPattern)
            d.setPattern(4, [pattern4] + dummyPattern)
            d.openHydraulicAnalysis()
            d.initializeHydraulicAnalysis()
            
            # Run and close analysis
            Series = d.getComputedHydraulicTimeSeries()
            d.closeHydraulicAnalysis()
            ModelTank1 = Series.Head[2, 302]
            ModelTank2 = Series.Head[2, 307]
            ModelTank3 = Series.Head[2, 304]
            Error = ((ModelTank1 - RealTank1) ** 2) + ((ModelTank2 - RealTank2) ** 2) + ((ModelTank3 - RealTank3) ** 2)
            
            # Store some of these new values
            demandFactors[whereMax] = [pattern1, pattern2, pattern3]
            pointsInSimplex[whereMax] = Error
            
#Results! (This will show zones 1 through 4)
print('The demand factors for hour 1 are as follows',demandFactors[0], pattern4)
#This is the error term on those results (how much the model tanks differ from the "real" tanks)
print('The error term for the first three zones are:', pointsInSimplex)

EPANET version 20200 loaded (EPyT version 1.0.7).
Input File DecemberEditsWhitesburg.inp loaded successfully.

The demand factors for hour 1 are as follows [10.004863968729467, 1.0053805794198976, 3.9838130993422958] 5.981118424519603
The error term for the first three zones are: [6.172217254573732e-08, 1.4826088225274103e-07, 1.5279326098156311e-07, 1.125654981436911e-07]


In [119]:
#Now we can test/ optimize the demand factors for the second hour

#Install the EPANET-Python toolkit (this is so we can process the hydraulic information)

from epyt import epanet
import numpy as np
import pandas as pd

# Pull in relevant EPANET file
d = epanet('C:/Users/apgi227/OneDrive - University of Kentucky/Documents/MATLAB/DecemberEditsWhitesburg.inp');

# Let's Extend The Box-Complex to the tank scenario
# Random Positive Demand Factors
demandFactors = [[2, 0.5, 3], [4, 1, 2], [3, 1.5, 0.5], [5, 1.2, 1]]

# Because I am only testing one hour and epanet requires demand factors for all 24 hours
# I will create 23 dummy factors
dummyPattern = [1] * 22

# Now here are the three functions (tanks) that we need to optimize
RealTank1 = 1485.18
RealTank2 = 1402.29
RealTank3 = 1468.87

# Initialize the error matrix (four points in box complex and their respective error)
pointsInSimplex = [1, 1, 1, 1]

# This starts the box-complex, here I specify that all of the points need to have a very small error term before it is finished.
# Essentially, converge on the solution. However this may run for a long time and loosening the tolerance will allow for quicker runs
while pointsInSimplex[0] >= 0.0000001 or pointsInSimplex[1] >= 0.0000001 or pointsInSimplex[2] <= 0.0000001 or pointsInSimplex[3] >= 0.0000001:
    # Evaluation of error for each of the four points in the simplex
    for i in range(4):
        # Create the demand patterns to place in the simulation
        pattern1 = demandFactors[i][0]
        pattern2 = demandFactors[i][1]
        pattern3 = demandFactors[i][2]
        
        # The fourth demand factor is determined by the first three - this is the total demand function.
        # This is the mass balance in the system which may be found by taking the basedemand for each of the points that are
        # specified by a certain demand pattern - multiplying those base demands by that factor, and summing them altogether.
        # This gives total demand. To satisfy conservation of mass, the fourth demand factor is determined by the other three.
        # 3, 226, and 48 represent the summed base demands for their respective zones. 605 represents the total demand of that specific hour.
        pattern4 = (605 - (3 * pattern1) - (226 * pattern2) - (48 * pattern3)) / 18
        
        # Run an initial sim to initialize the box complex
        d.setPattern(1, [10] + [pattern1] + dummyPattern)
        d.setPattern(2, [1] + [pattern2] + dummyPattern)
        d.setPattern(3, [4] + [pattern3] + dummyPattern)
        d.setPattern(4, [6] + [pattern4] + dummyPattern)
        d.openHydraulicAnalysis()
        d.initializeHydraulicAnalysis()
        
        # Run and close analysis
        Series = d.getComputedHydraulicTimeSeries()
        d.closeHydraulicAnalysis()
        ModelTank1 = Series.Head[3, 302]
        ModelTank2 = Series.Head[3, 307]
        ModelTank3 = Series.Head[3, 304]
        Error = ((ModelTank1 - RealTank1) ** 2) + ((ModelTank2 - RealTank2) ** 2) + ((ModelTank3 - RealTank3) ** 2)
        pointsInSimplex[i] = Error
        
    if pointsInSimplex[0] <= 0.0000001 or pointsInSimplex[1] <= 0.0000001 or pointsInSimplex[2] <= 0.0000001 or pointsInSimplex[3] <= 0.0000001:
        break
    else:
        # Find the worst point and create the centroid of the remaining points
        maxVal, whereMax = max(pointsInSimplex), pointsInSimplex.index(max(pointsInSimplex))
        logical = [i for i in range(4) if pointsInSimplex[i] != maxVal]
        
        ph = demandFactors[whereMax]
        centroid = [(demandFactors[logical[0]][j] + demandFactors[logical[1]][j] + demandFactors[logical[2]][j]) / 3 for j in range(3)]
       
        # Expand the worst point over the centroid of the remaining points
        newPoint = [(2.5 * centroid[j]) - (1.5 * ph[j]) for j in range(3)]
        
        # Contract if the New Point is less than 0
        while newPoint[0] < 0 or newPoint[1] < 0 or newPoint[2] < 0:
            newPoint = [(0.5 * newPoint[j]) + (0.5 * centroid[j]) for j in range(3)]
        
        # Now evaluate the new point to see if it is better than the old point
        pattern1 = newPoint[0]
        pattern2 = newPoint[1]
        pattern3 = newPoint[2]
        pattern4 = (605 - (3 * pattern1) - (226 * pattern2) - (48 * pattern3)) / 18
        d.setPattern(1, [10] + [pattern1] + dummyPattern)
        d.setPattern(2, [1] + [pattern2] + dummyPattern)
        d.setPattern(3, [4] + [pattern3] + dummyPattern)
        d.setPattern(4, [6] + [pattern4] + dummyPattern)
        d.openHydraulicAnalysis()
        d.initializeHydraulicAnalysis()
        
        # Run and close analysis
        Series = d.getComputedHydraulicTimeSeries()
        d.closeHydraulicAnalysis()
        ModelTank1 = Series.Head[3, 302]
        ModelTank2 = Series.Head[3, 307]
        ModelTank3 = Series.Head[3, 304]
        Error = ((ModelTank1 - RealTank1) ** 2) + ((ModelTank2 - RealTank2) ** 2) + ((ModelTank3 - RealTank3) ** 2)
        
        # If the new error is less than the old store the value
        if Error < maxVal:
            demandFactors[whereMax] = [pattern1, pattern2, pattern3]
            pointsInSimplex[whereMax] = Error
            
        # If the error is worse than the new point, contract the worst point towards the centroid
        else:
            newPoint = [(0.5 * ph[j]) + (0.5 * centroid[j]) for j in range(3)]
            
            # Evaluate the new point
            pattern1 = newPoint[0]
            pattern2 = newPoint[1]
            pattern3 = newPoint[2]
            pattern4 = (605 - (3 * pattern1) - (226 * pattern2) - (48 * pattern3)) / 18
            d.setPattern(1, [10] + [pattern1] + dummyPattern)
            d.setPattern(2, [1] + [pattern2] + dummyPattern)
            d.setPattern(3, [4] + [pattern3] + dummyPattern)
            d.setPattern(4, [6] + [pattern4] + dummyPattern)
            d.openHydraulicAnalysis()
            d.initializeHydraulicAnalysis()
            
            # Run and close analysis
            Series = d.getComputedHydraulicTimeSeries()
            d.closeHydraulicAnalysis()
            ModelTank1 = Series.Head[3, 302]
            ModelTank2 = Series.Head[3, 307]
            ModelTank3 = Series.Head[3, 304]
            Error = ((ModelTank1 - RealTank1) ** 2) + ((ModelTank2 - RealTank2) ** 2) + ((ModelTank3 - RealTank3) ** 2)
            
            # Store some of these new values
            demandFactors[whereMax] = [pattern1, pattern2, pattern3]
            pointsInSimplex[whereMax] = Error

            
#Results! (This will show zones 1 through 4)
print('The demand factors for hour 2 are as follows',demandFactors[0], pattern4)
#This is the error term on those results (how much the model tanks differ from the "real" tanks)
print('The error term for the first three zones are:', pointsInSimplex[0])     
           

EPANET version 20200 loaded (EPyT version 1.0.7).
Input File DecemberEditsWhitesburg.inp loaded successfully.

The demand factors for hour 2 are as follows [8.023428753175525, 0.5091000896724001, 6.0154393808366144] 9.842870313313988
The error term for the first three zones are: 1.5999708047201377e-07


'Bartesta_Tank'